# Spin Classification Model

A deep learning model to classify quantum spin configurations

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

### Loading Datasets

You can load different datasets by changing the dataset path for testing, trainging, and benchmarking

In [ ]:
# Set these variables to the appropriate directory paths

test_dir = '../data/binary_class/test/'
train_dir = '../data/binary_class/train/'

# Set these variables to the appropriate directory paths
benchmark_dir = '../data/temp_class/test/'

In [ ]:
# data transforms
dset_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()])

#create datasets using ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=dset_transform)
test_data = datasets.ImageFolder(test_dir, transform=dset_transform)
benchmark_data = datasets.ImageFolder(benchmark_dir, transform=dset_transform)

# create dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

## Architecture

This is the basic architecture of the model. The following parameters can be set:

* `input_size`: The dimensions of the input images. Default: `[25, 25]`
* `hidden_layers`: The number of hidden layers in the model. Default: 5
* `layer_size`: The size of each hidden layer. Default: 128
* `conv`: Boolean to determine if the network should be fully connected or convolutional. Default: False
* `num_classes`: The number of output classes. Leave as 2 for binary classification. Otherwise set to number of temperature values for which data is available. Default: 2


In [ ]:
input_size = [25,25]
hidden_layers = 5
layer_size = 128
conv = True # True for convolutional, False for fully connected
num_classes = 2

In [ ]:
def linear_block(layer_size, num_layers):
    layers = []
    for i in range(num_layers):
        layers.append(nn.Linear(layer_size, layer_size))
        layers.append(nn.ReLU())
    return layers
    

In [ ]:
def conv_block(layer_size, num_layers):
    layers = []
    for i in range(num_layers):
        layers.append(nn.Conv2d(layer_size, layer_size, kernel_size=3, stride=1, padding=1))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return layers

In [ ]:
class SpinClassifier(nn.Module): 
    def __init__(self, input_size, hidden_layers, layer_size, num_classes, conv):
        super(SpinClassifier, self).__init__()
        self.conv = conv
        self.input_dim = np.prod(input_size)
        self.output_dim = num_classes
        self.num_layers = hidden_layers
        self.layer_size = layer_size
        self.input_layer = [nn.Linear(self.input_dim, self.layer_size), nn.ReLU()]
        self.output_layer = [nn.Linear(self.layer_size, self.output_dim)]
        if self.conv:
            self.layers = conv_block(self.layer_size, self.num_layers)
        else:
            self.layers = linear_block(self.layer_size, self.num_layers)
        
        self.model = nn.Sequential(*self.input_layer, *self.layers, *self.output_layer)


        
    def forward(self, x):
        x.view(-1, self.input_dim)
        return self.model(x)
        